# Autoencoders in Keras

* [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
* [A Tutorial on Deep Learning PDF](http://ai.stanford.edu/~quocle/tutorial2.pdf)